# Beaver Triples and multiplication

In this notebook we will show a tecnique to find the multiplication of two shared numbers using randomly generated shares.

We've seen that we can do multiplication in Shamir secret sharing only if we have $2t$ parties that can reconstruct the secret. In this case we will generate some random numbers called Beaver triples to help us perform the calculation. 

## Secretly computing one multiplication

In the first example we will try to secretly multipy two integers $x$ and $y$ that are owned by Ausiàs and Bernat, respectively. They want to compute $x*y$ without each other (or nobody else) know what is $x$ or $y$ actually. To this end we will use some computational parties $n$ that will carry the computation.

In [1]:
from random import seed, randrange
seed(3)

x = 5
y = 21

print(f"Ausias has a secret value ausias_secret={x}")
print(f"Bernat has a secret value bernat_secret={y}")

Ausias has a secret value ausias_secret=5
Bernat has a secret value bernat_secret=21


In [2]:
from crypt import GeneratePrimeGeneratorPair
from smpc import BeaverTriplesGenerator

n = 2 # number of parties involved
k = 1 # number of multiplications to perform
bits = 16 # bits of the calculation

p, _ = GeneratePrimeGeneratorPair(bits)

print(f"Multiplication will be performed using {n} parties")
print(f"Prime number of {bits} bits randomly chosen: {p}")

Multiplication will be performed using 2 parties
Prime number of 16 bits randomly chosen: 63587


First thing Ausias and Bernat have to do is to split into additive shares their values:

In [3]:
def share(secret, n, p):
    # generate n random shares
    shares = [randrange(p) for _ in range(n-1)]
    shares.append((secret-sum(shares))%p)
    return shares

def reconstruct(shares, p):
    return (sum(shares)%p)

x_shares = share(x, n, p)
y_shares = share(y, n, p)

print(f"shares for Ausias value:\n\t{x_shares}")
print(f"shares for Bernat value:\n\t{y_shares}")

shares for Ausias value:
	[5239, 58353]
shares for Bernat value:
	[22066, 41542]


In [4]:
print(f"Ausias value reconstructed: {reconstruct(x_shares, p)}")
print(f"Bernat value reconstructed: {reconstruct(y_shares, p)}")

Ausias value reconstructed: 5
Bernat value reconstructed: 21


We have to generate the Beaver triples, these are a triplet of shared values $[a]$, $[b]$, $[c]$ such that when reconstructed $c=a\times b$. Check the function BeaverTriplesGenerator for details on the generation of such numbers

In [5]:
generator = BeaverTriplesGenerator(n, p)
a, b, c = generator.GenerateKBatches(k)

print("Showing shares of a, b and c: \n")
print(f"[a] = {a}\n")
print(f"[b] = {b}\n")
print(f"[c] = {c}")

Showing shares of a, b and c: 

[a] = [[48843, 15605]]

[b] = [[7917, 23608]]

[c] = [[11260, 43703]]


Let's check that the triple generation is done correctly by checking that:

$$a*b = c$$

so we reconstruct a, b and c modulo p and then check the equality

In [6]:
a_reconstructed = reconstruct(a[0], p)
b_reconstructed = reconstruct(b[0], p)
c_reconstructed = reconstruct(c[0], p)

assert a_reconstructed*b_reconstructed%p==c_reconstructed%p
print(f"cumulative sum of a times cumulative sum of b modulo p {a_reconstructed*b_reconstructed%p}")
print(f"cumulative sum of c modulo p {c_reconstructed%p}")
print("They are equal!")

cumulative sum of a times cumulative sum of b modulo p 54963
cumulative sum of c modulo p 54963
They are equal!


The shares $[a]$, $[b]$, $[c]$ have to be split into the workers. We also print the shared value of $x$ and $y$ to show what each party gets.

In [7]:
for i, (x_, y_, a_, b_, c_) in enumerate(zip(x_shares, y_shares, a[0], b[0], c[0])):
    print(f"Party {i}:\n\tx={x_}\n\ty={y_}\n\ta={a_}\n\tb={b_}\n\tc={c_}\n\t")

Party 0:
	x=5239
	y=22066
	a=48843
	b=7917
	c=11260
	
Party 1:
	x=58353
	y=41542
	a=15605
	b=23608
	c=43703
	


Now each party has to calculate:

$$[\epsilon]=[x]-[a]$$
$$[\delta]=[y]-[b]$$

In [8]:
e, d = [], []

for x_, y_, a_, b_, c_ in zip(x_shares, y_shares, a[0], b[0], c[0]):
    e.append((x_ - a_)%p)
    d.append((y_ - b_)%p)
    
epsilon = reconstruct(e, p)
delta = reconstruct(d, p)

print(f"They open epsilon and delta: \n\nepsilon={epsilon}\ndelta={delta}")

They open epsilon and delta: 

epsilon=62731
delta=32083


Recall that $\epsilon$ and $\delta$ are one time padded (we use a different random Beaver triple every time we run a multiplication) and therefeore do not provide any information about $x$ and $y$ the values that Ausias and Bernat want to multiply using third party helpers.

Now each party has to calculate:

$$[z] = [c] + \epsilon [b] + \delta[a] + \epsilon\delta$$


WARNING! Look out on this step! $\epsilon\delta$ is now a public value and just one party has to add it to the calculation. We do it in the first one by default

In [9]:
z = []
for x_, y_, a_, b_, c_ in zip(x_shares, y_shares, a[0], b[0], c[0]):
    z_ = (c_ + epsilon*b_ + delta*a_)%p
    z.append(z_)

# adding public value to the first client only
z[0] += epsilon*delta

Now everybody make public their shares so they can reconstruct

In [10]:
print(f"The result of multiplicate {x} and {x} secret values is {reconstruct(z, p)}")
assert x*y==reconstruct(z, p), "Something went wrong, reconstruction was not correct"

The result of multiplicate 5 and 5 secret values is 105


Even though the following calculation is not secret it is nice to see that if we reconstruct all the values and perform the same calculation as above we get the same result.

In [11]:
zp = (reconstruct(c[0], p) + epsilon*reconstruct(b[0], p) + delta*reconstruct(a[0], p) + epsilon*delta)%p
assert zp%p==reconstruct(z, p), "Something went wrong, reconstruction was not correct"

Or equivalently reconstructing on the shares of x and y:

In [12]:
zp = reconstruct(c[0], p) + delta*reconstruct(x_shares, p)%p + epsilon*reconstruct(y_shares, p)%p - epsilon*delta%p
assert zp%p==reconstruct(z, p), "Something went wrong, reconstruction was not correct"


So is this really secure?. Let's analyze... The values of the triplets contain no information about the secret values $x$ and $y$ and are suposed to be uniromly random between the range of 0 to $p$. Therefore wen we calculate $\epsilon$ and $\delta$ we are padding with a random value and so each share of $\epsilon$ and $\delta$ contain no information about the actual value of $x$ and $y$ so the parties can share their corresponding shares for $\epsilon$ and $\delta$. 


## Several multiplications

Now imagine that the holders of the private values $x$ and $y$ want to outsource the calculation of several values, let's say 10 to 4 parties. We have to generate 10 beaver triplets of lenght 4. Let's code this!

In [13]:
from random import seed, randrange
from crypt import GeneratePrimeGeneratorPair
from smpc import BeaverTriplesGenerator

seed(5)

n = 4 # number of parties involved
k = 10 # number of multiplications to perform
bits = 32 # bits of the calculation

p, _ = GeneratePrimeGeneratorPair(bits)

x = [randrange(p//4) for _ in range(k)]
y = [randrange(p//4) for _ in range(k)]

print(f"Prime number of {bits} bits randomly chosen: {p}\n")
print(f"x = {x}")
print(f"y = {y}\n")

print(f"Multiplication will be performed using {n} parties")
print(f"We can perform {k} multiplications")


Prime number of 32 bits randomly chosen: 4226052217

x = [171382766, 335484322, 1034632707, 915970885, 448268701, 710108163, 302918109, 451838982, 882340072, 899441922]
y = [635070364, 725609445, 494935713, 364905823, 982022104, 785802611, 562876344, 542403849, 352021502, 764032359]

Multiplication will be performed using 4 parties
We can perform 10 multiplications


In [14]:
x_shares = [share(x_, n, p) for x_ in x]
y_shares = [share(y_, n, p) for y_ in y]

print(f"x shares: \n\t")
for x_s in x_shares:
    print(f"\t{x_s}")
    
print(f"y shares: \n\t")
for y_s in y_shares:
    print(f"\t{y_s}")

x shares: 
	
	[2960040999, 2638725046, 3389060587, 3861712785]
	[3418869866, 61026012, 684227583, 397413078]
	[2477863492, 2124033828, 2800158410, 2084681411]
	[5200585, 2315099732, 3396513666, 3651261336]
	[2073382968, 3313593901, 2000151613, 1513244653]
	[3595306181, 1602804837, 185424399, 3778677180]
	[976545007, 2298812163, 8900742, 1244712414]
	[2055551344, 4171603642, 513230824, 2163557606]
	[3164887442, 870048867, 725786741, 347669239]
	[4184824778, 733871712, 3662057698, 770792168]
y shares: 
	
	[2178846392, 1022962142, 2860127913, 3025238351]
	[3561047244, 1715454038, 2795991898, 1105220699]
	[439049416, 1653147711, 1793752823, 835037980]
	[2769103316, 3708372235, 983575982, 1355958724]
	[2772820510, 1175782748, 3951769298, 1533753982]
	[43340798, 880567911, 2820272352, 1267673767]
	[1013079011, 2497131583, 287508572, 991209395]
	[3192928261, 3998657813, 2163732437, 3865241989]
	[1602723052, 1867731291, 2500476193, 2833195400]
	[1316040632, 957809239, 3158144681, 3784142241]


In [15]:
generator = BeaverTriplesGenerator(n, p)
a, b, c = generator.GenerateKBatches(k)

print("Showing shares of a, b and c: \n")
print(f"[a] = ")
for a_s in a:
    print(f"\t{a_s}")

print(f"[b] = ")
for b_s in b:
    print(f"\t{b_s}")
    
print(f"[c] = ")
for c_s in c:
    print(f"\t{c_s}")
    
    
print(f"\nnumber of multiplications to perform is : {len(a)}")

Showing shares of a, b and c: 

[a] = 
	[1957249673, 2254077750, 3297105215, 1056760732]
	[395485659, 3739766301, 999360773, 236896232]
	[2827189904, 2408203629, 3791262365, 2259085819]
	[2499862764, 1075250475, 3075445279, 2671770808]
	[3608950050, 692035623, 1570366022, 645950124]
	[2694167901, 1237044081, 782197768, 3014893532]
	[25785821, 3010546550, 2362099281, 3504365697]
	[3803214799, 342875159, 2094985100, 1104926244]
	[3803352435, 2051571744, 1136892690, 3047917330]
	[2306181790, 1771393359, 733090959, 2026787461]
[b] = 
	[864717120, 2978299693, 2952975892, 2832539761]
	[3939121783, 1814821073, 3406089661, 2568009658]
	[3546103788, 3064892670, 3157225860, 2548217026]
	[2928783976, 3723505140, 3304518172, 4127793019]
	[2719862558, 406764726, 3043254524, 2221879130]
	[29579488, 2337280713, 639444605, 1837647468]
	[2004056478, 4113526903, 3559062689, 1965889497]
	[646052392, 1578755279, 29178817, 1716547518]
	[3450272090, 2454736490, 1938153019, 3860147564]
	[1688840353, 18318225

In [16]:
epsilon, delta = [], []
# looping over multiplications
for i in range(k):
    e, d = [], []
    for x_, y_, a_, b_, c_ in zip(x_shares[i], y_shares[i], a[i], b[i], c[i]):
        e.append((x_ - a_)%p)
        d.append((y_ - b_)%p)
    
    epsilon.append(reconstruct(e, p))
    delta.append(reconstruct(d, p))
    
print(f"They open epsilon and delta: \n\nepsilon={epsilon}\ndelta={delta}")

They open epsilon and delta: 

epsilon=[58293830, 3416079791, 2427047641, 45745993, 2383071316, 1433909315, 4078277411, 1557942114, 3520762524, 2514092787]
delta=[3684694549, 1675723921, 856653020, 3184514384, 1042365600, 167902554, 1598497428, 797922060, 1326868990, 827130240]


In [17]:
z = []
for i in range(k):
    z_tmp = []
    for x_, y_, a_, b_, c_ in zip(x_shares[i], y_shares[i], a[i], b[i], c[i]):
        z_ = (c_ + epsilon[i]*b_ + delta[i]*a_)%p
        z_tmp.append(z_)
        
    # adding public value to the first client only
    z_tmp[0] += epsilon[i]*delta[i]
    z.append(z_tmp)

In [18]:
print(z)

[[214794960964287581, 1299596488, 3524032757, 4102672674], [5724406623834744671, 2431305481, 587980058, 957238475], [2079137695471337508, 2136170787, 1454087343, 1233055626], [145678773715715969, 725725219, 2749086242, 4000108859], [2484031564507987983, 1454987998, 490613773, 1006931053], [240757036954823877, 1697780883, 3345746177, 2181188042], [6519115956152937177, 1741132950, 1371396328, 2819467901], [1243116384520319745, 3874444826, 3390020310, 2033888922], [4671590616328410370, 349905084, 3191634670, 360875086], [2079482174408344051, 3738211994, 1923943103, 2424188983]]


In [19]:
for i in range(k):
    print(f"multiplication of {x[i]} times {y[i]} is {x[i]*y[i]%p}, we calculated secretly {reconstruct(z[i], p)}")
    assert x[i]*y[i]%p == reconstruct(z[i], p), "Something went wrong"

multiplication of 171382766 times 635070364 is 201087304, we calculated secretly 201087304
multiplication of 335484322 times 725609445 is 3058084736, we calculated secretly 3058084736
multiplication of 1034632707 times 494935713 is 247014640, we calculated secretly 247014640
multiplication of 915970885 times 364905823 is 3813151306, we calculated secretly 3813151306
multiplication of 448268701 times 982022104 is 971965664, we calculated secretly 971965664
multiplication of 710108163 times 785802611 is 3089304220, we calculated secretly 3089304220
multiplication of 302918109 times 562876344 is 2396237340, we calculated secretly 2396237340
multiplication of 451838982 times 542403849 is 778287945, we calculated secretly 778287945
multiplication of 882340072 times 352021502 is 2049008670, we calculated secretly 2049008670
multiplication of 899441922 times 764032359 is 26634969, we calculated secretly 26634969


## Conclusions

We've seen how by generating random triples we can achieve secure multiplication. However one must take care on who provides these random triples. In general we would have a "crypto provider", that is, a machine that provides the triples but is not active on the protocol (does not compute/hold any shares). The crypto provider calculates the triples on an offline phase and can store them to send to the parties when they require to do a computation. This party can be honest but courious (follows the protocol, i.e. provides correct triples).